<img style="float: left;" src="D2Klogo.png" width="20%"> 

<center>

# Welcome to COMP 680 #
### Statistics for Computing and Data Science ###

### Week 10 Stochastic Processes ###

In [ ]:
### standard imports
import numpy as np
import pandas as pd

#import scipy linear algebra
import scipy.linalg as la

%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["patch.force_edgecolor"] = True
import seaborn as sns

import random
random.seed(2023)

## Simulate Random Walk

In [ ]:
def generate_random_walk(steps):
    X = [0]
    for i in range(steps):
        step = np.random.uniform(low=-1, high=1)
        X.append(X[i] + step)
    return X

In [ ]:
def generate_random_walk_bound(steps, drift=-0.4, min_value=-10):
    X = [0]
    for i in range(steps):
        step = np.random.uniform(low=-1, high=1)
        X.append(max(X[i] + drift, min_value) + step)
    return X

In [ ]:
plt.figure(figsize=(12, 8))

A = generate_random_walk(1000)
B = generate_random_walk_bound(1000)

ax = plt.subplot(2, 1, 1)
ax.plot(np.arange(0, len(A)), A)
ax.set_title('Not Stationary')

ax = plt.subplot(2, 1, 2)
ax.plot(np.arange(0, len(B)), B)
ax.set_title('Stationary')

plt.tight_layout()
plt.show()

## Simulate Markov Chain

Consider a two state Markov chain with states $\mathcal{X} = \{ 1, 2 \}$ and transition matrix

$$ P = \begin{bmatrix}
1 - a & a \\
b & 1 - b
\end{bmatrix} $$

where $0 < a < 1$ and $0 < b < 1$.  Can you show that the stationary distribution is:

$$ 
\pi_{stat} = \left[ \frac{b}{a + b}, \frac{a}{a + b} \right]
$$

When simulate the chain, we will calculate the empirical probabilty:

$$
\hat{p}_n(1) = \frac{1}{n} \sum_{i=1}^n I(X_i = 1)
\quad \text{and} \quad
\hat{p}_n(2) = \frac{1}{n} \sum_{i=1}^n I(X_i = 2)
$$

i.e, the proportion of times the chain is in state 1 and state 2. We should compare the values of $\hat{p}_n(1)$ and $\hat{p}_n(2)$ to the true value of $\mu_{stat}$.

In [ ]:
# define the parameters
a = 0.3
b = 0.1
P = np.array([[1 - a, a], [b, 1 - b]])
pi_stat = np.array([b/(a+b), a/(a+b)])
P

In [ ]:
# draw a random variable ~ Uni[0,1]
np.random.rand()

In [ ]:
# Do a *single* simulation starting from, say, state 1

def generate_series(steps, initial_state=1):
    
    # start with initial state: this is time 0
    X = [initial_state]
    # a for loop to simulate a chain
    # start with time 1
    for i in np.arange(1, steps): 
        # what is the next state:
        # current state is k = X[i-1], want transition prob from P[k-1, ]
        # but since there are only two columns, only need to look at first column
        # trans prob: p = P[X[i-1]-1, 0]
        next_state = 1 if np.random.rand() < P[X[i-1]-1, 0] else 2
        # this is the same as sample a value from Bernoulli(p), but failure is 2 instead of 0
        X.append(next_state)
        
    return X

- [`np.cumsum`](https://numpy.org/doc/stable/reference/generated/numpy.cumsum.html)

In [ ]:
# generate the chain
X = np.array(generate_series(steps=10000))
# define time steps
T = np.arange(1, len(X)+1)
# this is p_n(1)_hat: an array of prob with len = stpes
p1 = np.cumsum(X == 1) / T
# this is p_n(2)_hat: an array of prob with len = stpes
p2 = np.cumsum(X == 2) / T

In [ ]:
p1

In [ ]:
# plot truth vs. empirical estimate
plt.figure(figsize=(12, 8))
plt.plot(T, p1, label=r'$\hat{p}_n(1)$')
plt.axhline(y=pi_stat[0], color='green', label=r'$p(1)$')
plt.plot(T, p2, label=r'$\hat{p}_n(2)$')
plt.axhline(y=pi_stat[1], color='red', label=r'$p(2)$')
plt.legend()
plt.show()

Let

$$ P = \begin{bmatrix}
0.40 & 0.50 & 0.10 \\
0.05 & 0.70 & 0.25 \\
0.05 & 0.50 & 0.45
\end{bmatrix}$$

How can we find the stationary distribution $\pi_{stat}$?

We will use [scipy.linalg.solve](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.solve.html#scipy.linalg.solve) to solve linear systems.

In [ ]:
def stationary_dist(P):
    ''' Given a transition matrix P for a Markov chain
     return the stationary distribution
     P should be a square n by n matrix
     solve for: P*pi_stat = pi_stat or (P-I)*pi_stat = 0
    '''
    n = P.shape[0]
    # Pch = P - I 
    Pch = P - np.eye(n)
    # replace column 0 with all ones
    Pch[:,0] = np.ones(n)
    # rhs = [1, 0, ... 0]
    rhs = np.zeros(n)
    rhs[0] = 1
    # solve the system 
    # NOTE: solver wants Ax = b
    # so transform sytem w transpos
    ans = la.solve(Pch.T,rhs)
    
    return ans.T

In [ ]:
P = np.array([[0.4, 0.5, 0.1], [0.05, 0.7, 0.25], [0.05, 0.5, 0.45]])
P

In [ ]:
pi_stat = stationary_dist(P)
pi_stat

In [ ]:
# check
pi_stat @ P

#### An oversimplified application to marketing and customer churning. 

Suppose there are four big market players in the cell phone services industry. Current market share among the four competitors: A, B, C and D is a relatively stable distribution of $[.55, 0.2, 0.1, 0.15]$. The second biggest competitor B has just launched a new premium product. The biggest competitor A of course want to understand what will happen to their market share if they do not intervene. 

Suppose we do a market survey to understand how consumers are moving between the different providers with the new premium product launched by B, and from there we can construct a transition probability matrix as follows:

$$ P = \begin{bmatrix}
0.9262 & 0.0385 & 0.01 & 0.0253 \\
0.01 & 0.94 & 0.01 & 0.04 \\
0.005 & 0.035 & 0.92 & 0.04 \\
0.035 & 0.035 & 0.035 & 0.895 \\
\end{bmatrix}$$



In [ ]:
P = np.array([[0.9262, 0.0385, 0.01, 0.0253],
              [0.01, 0.94, 0.01, 0.04],
              [0.005, 0.035, 0.92, 0.04],
              [0.035, 0.035, 0.035, 0.895]])
pi_stat = stationary_dist(P)
pi_stat